
# Imports

In [ ]:
import math
import time
import random
import numpy as np
import threading
import os
import json
import csv

# Lecture du fichier

In [ ]:

def calculate_distance_file(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def read_tsp_file(file_path):
    coordinates = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        node_coord_section_index = lines.index('NODE_COORD_SECTION\n')
        for line in lines[node_coord_section_index + 1:]:
            if line.strip() == 'EOF':
                break
            parts = line.strip().split()
            node_id = int(parts[0])
            x, y = map(float, parts[1:])
            coordinates[node_id] = (x, y)
    return coordinates


def create_distance_matrix(coordinates):
    num_cities = len(coordinates)
    distance_matrix = [[0] * num_cities for _ in range(num_cities)]

    for i in range(1, num_cities + 1):
        for j in range(1, num_cities + 1):
            distance_matrix[i - 1][j - 1] = calculate_distance_file(coordinates[i], coordinates[j])

    return distance_matrix

# Example usage
file_path = "att48.tsp"
coordinates = read_tsp_file(file_path)
distances = create_distance_matrix(coordinates)



# NEAREST NEIGHBOUR

In [ ]:
def nearest_neighbor(tour, visited, num_cities, distances):
    start_time = time.time()
    # If the tour is empty, initialize it with the nearest neighbor algorithm
    if len(tour) == 0:

        visited = [False] * num_cities
        # Initialize variables to track the minimum distance and the cities
        min_distance = float('inf')
        start_city = None
        next_city = None

        # Step 1: Find the shortest edge connecting two cities
        for i in range(num_cities):
            for j in range(num_cities):
                if (i != j) and (distances[i][j] < min_distance):
                    min_distance = distances[i][j]
                    start_city = i
                    next_city = j

        # Add the starting city and its nearest neighbor to the tour
        tour.append(start_city)
        tour.append(next_city)
        visited[start_city] = True
        visited[next_city] = True

    # If there are still cities to visit
    elif len(tour) < num_cities:
        # Step 2: Find the nearest neighbor of the current city
        current_city = tour[-1]
        min_distance = float('inf')
        next_city = None

        for i in range(num_cities):
            # Check if the city has not been visited and if its distance is shorter
            if not visited[i] and distances[current_city][i] < min_distance:
                min_distance = distances[current_city][i]
                next_city = i

        # Step 3: Add the nearest city to the tour
        tour.append(next_city)
        visited[next_city] = True

    # If all cities have been visited, return to the starting city to complete the tour
    else:
        tour.append(tour[0])

    cost = calculate_tour_cost(tour,distances)
    end_time = time.time()
    elapsed_time = end_time - start_time

    return tour, visited, cost, elapsed_time

def calculate_tour_cost(tour, distances):
    cost = 0
    for i in range(len(tour) - 1):
        cost += distances[tour[i]][tour[i+1]]
    return cost
"""
distances = [
    [0, 29, 20, 21],
    [29, 0, 15, 17],
    [20, 15, 0, 28],
    [21, 17, 28, 0]
]
"""
NumberofCities = len(distances)
tour = []
visited = []

while len(tour) <= NumberofCities:
    tour, visited, cost, elapsed_time = nearest_neighbor(tour, visited, NumberofCities, distances)

print(f"Elapsed Time: {elapsed_time} seconds")
print("Optimal Tour:", tour)
print("Tour Cost:", cost)


"""
tour = [1, 8, 38, 31, 44, 18, 7, 28, 6, 37,
    19, 27, 17, 43, 30, 36, 46, 33, 20, 47,
    21, 32, 39, 48, 5, 42, 24, 10, 45, 35,
    4, 26, 2, 29, 34, 41, 16, 22, 3, 23,
    14, 25, 13, 11, 12, 15, 40, 9,1]
calculate_tour_cost(tour, distances)
"""

Elapsed Time: 2.86102294921875e-05 seconds
Optimal Tour: [18, 36, 5, 29, 26, 42, 16, 27, 6, 17, 35, 43, 30, 37, 8, 0, 7, 21, 15, 2, 22, 10, 11, 14, 32, 45, 39, 19, 46, 20, 12, 24, 13, 33, 40, 28, 4, 47, 38, 31, 23, 9, 41, 25, 3, 34, 44, 1, 18]
Tour Cost: 42754.52629466549


'\ntour = [1, 8, 38, 31, 44, 18, 7, 28, 6, 37,\n    19, 27, 17, 43, 30, 36, 46, 33, 20, 47,\n    21, 32, 39, 48, 5, 42, 24, 10, 45, 35,\n    4, 26, 2, 29, 34, 41, 16, 22, 3, 23,\n    14, 25, 13, 11, 12, 15, 40, 9,1]\ncalculate_tour_cost(tour, distances)\n'

# NEAREST AND LONLIEST NEIGHBOUR



In [ ]:
def calculate_loneliness(distances):
    distset = [sum(row) for row in distances]
    #print(distset)
    min_distset = min(distset)
    max_distset = max(distset)
    average_distset = (max_distset + min_distset) / 2

    for i in range(len(distset)):
        if distset[i] > average_distset:
            distset[i] = average_distset - (distset[i] - average_distset)
        else:
            distset[i] = average_distset + (average_distset - distset[i])

    return distset

def create_new_distance_matrix(distances, lonliness):
    num_cities = len(distances)
    new_distances = [[0] * num_cities for _ in range(num_cities)]

    for i in range(num_cities):
        for j in range(num_cities):
            new_distances[i][j] = ((num_cities * distances[i][j]) + lonliness[j]) / 2

    return new_distances

def nearest_loneliest_neighbor(tour, visited, num_cities, old_distances, distances):

    start_time = time.time()

    if len(tour) == 0:
        visited = [False] * num_cities
    # Step 1: Find the shortest edge and take it as the first tour edge
        min_distance = float('inf')
        start_city = None
        next_city = None

        for i in range(num_cities):
            for j in range(num_cities):
                if (i != j) and  (distances[i][j] < min_distance):
                    min_distance = distances[i][j]
                    start_city = i
                    next_city = j

        tour.append(start_city)
        tour.append(next_city)
        visited[start_city] = True
        visited[next_city] = True

    elif len(tour) < num_cities:
        # Step 2: Find the nearest neighbor of the current city
        current_city = tour[-1]
        min_distance = float('inf')
        next_city = None

        # Step 2: Find the nearest and loneliest neighbor
        for i in range(num_cities):
            if not visited[i] and distances[current_city][i] < min_distance:
                min_distance = distances[current_city][i]
                next_city = i

        # Step 3: Add the nearest city to the tour
        tour.append(next_city)
        visited[next_city] = True
        current_city = next_city

    # Step 4: Return to the starting city
    else:
        tour.append(tour[0])

    cost = calculate_tour_cost(tour, old_distances)
    end_time = time.time()
    elapsed_time = end_time - start_time

    return tour, visited, cost, elapsed_time

"""
def calculate_tour_cost(tour, distances):
    cost = 0
    for i in range(len(tour) - 1):
        cost += distances[tour[i]][tour[i+1]]
    return cost


distances = [
    [0, 29, 20, 21],
    [29, 0, 15, 17],
    [20, 15, 0, 28],
    [21, 17, 28, 0]
]
"""

NumberofCities = len(distances)
tour = []
visited = []

# Step 2: Calculate loneliness of each city
lonliness = calculate_loneliness(distances)

# Step 3: Create new distance matrix
new_distances = create_new_distance_matrix(distances, lonliness)

#tour, visited, cost, elapsed_time = nearest_neighbor(tour, visited, NumberofCities, new_distances)

while len(tour) <= NumberofCities/2:
    tour, visited, cost, elapsed_time = nearest_loneliest_neighbor(tour, visited, NumberofCities, distances ,new_distances)

while len(tour) <= NumberofCities:
    tour, visited, cost, elapsed_time = nearest_neighbor(tour, visited, NumberofCities, distances)


print(f"Elapsed Time: {elapsed_time} seconds")
print("Optimal Tour:", tour)
print("Tour Cost:", cost)

Elapsed Time: 6.67572021484375e-06 seconds
Optimal Tour: [25, 3, 34, 44, 9, 23, 41, 1, 28, 4, 47, 38, 31, 20, 46, 19, 32, 45, 43, 17, 6, 27, 5, 36, 18, 26, 42, 29, 16, 35, 30, 37, 8, 0, 7, 21, 15, 2, 22, 10, 11, 14, 39, 12, 24, 13, 33, 40, 25]
Tour Cost: 38045.08244163059


# DOUBLE-ENDED NEAREST NEIGHBOUR

In [ ]:
def double_ended_nearest_neighbor(tour, visited, num_cities, distances):
    start_time = time.time()
    # If the tour is empty, initialize it with the nearest neighbor algorithm
    if len(tour) == 0:

        visited = [False] * num_cities
        # Initialize variables to track the minimum distance and the cities
        min_distance = float('inf')
        start_city = None
        next_city = None

        # Step 1: Find the shortest edge connecting two cities
        for i in range(num_cities):
            for j in range(num_cities):
                if (i != j) and (distances[i][j] < min_distance):
                    min_distance = distances[i][j]
                    start_city = i
                    next_city = j

        # Add the starting city and its nearest neighbor to the tour
        tour.append(start_city)
        tour.append(next_city)
        visited[start_city] = True
        visited[next_city] = True

    # If there are still cities to visit
    elif len(tour) < num_cities:


        # Step 2: Consider the nodes closer to each of the route’s ends
        # and add to the tour the one closer to the respective endpoint
        start_min_distance = float('inf')
        end_min_distance = float('inf')
        start_city = tour[0]
        end_city = tour[-1]

        for i in range(num_cities):
            if not visited[i]:
                if distances[i][tour[0]] < start_min_distance:
                    start_min_distance = distances[i][tour[0]]
                    start_city = i
                if distances[i][tour[-1]] < end_min_distance:
                    end_min_distance = distances[i][tour[-1]]
                    end_city = i

        # Choose the closer city and add it to the tour
        if start_min_distance <= end_min_distance:
            tour.insert(0, start_city)
            visited[start_city] = True
        else:
            tour.append(end_city)
            visited[end_city] = True
    else:
    # Step 4: Return to the starting node by adding it to the end of the route
      tour.append(tour[0])


    cost = calculate_tour_cost(tour, distances)
    end_time = time.time()
    elapsed_time = end_time - start_time

    return tour, visited, cost, elapsed_time

"""
def calculate_tour_cost(tour, distances):
    cost = 0
    for i in range(len(tour) - 1):
        cost += distances[tour[i]][tour[i + 1]]
    return cost


distances = [
    [0, 29, 20, 21],
    [29, 0, 15, 17],
    [20, 15, 0, 28],
    [21, 17, 28, 0]
]
"""
NumberofCities = len(distances)
tour = []
visited = []

while len(tour) <= NumberofCities:
    tour, visited, cost, elapsed_time = double_ended_nearest_neighbor(tour, visited, NumberofCities, distances)

print(f"Elapsed Time: {elapsed_time} seconds")
print("Optimal Tour:", tour)
print("Tour Cost:", cost)


Elapsed Time: 6.9141387939453125e-06 seconds
Optimal Tour: [16, 42, 26, 18, 36, 5, 29, 27, 6, 17, 35, 43, 30, 37, 8, 0, 7, 21, 15, 2, 22, 10, 11, 14, 32, 45, 39, 19, 46, 20, 12, 24, 13, 33, 40, 28, 4, 47, 38, 31, 23, 9, 41, 25, 3, 34, 44, 1, 16]
Tour Cost: 42156.22071972136


# DOUBLE-ENDED NEAREST AND LONELIEST NEIGHBOUR

In [ ]:
"""
def calculate_loneliness(distances):
    distset = [sum(row) for row in distances]
    min_distset = min(distset)
    max_distset = max(distset)
    average_distset = (max_distset + min_distset) / 2

    for i in range(len(distset)):
        if distset[i] > average_distset:
            distset[i] = average_distset - (distset[i] - average_distset)
        else:
            distset[i] = average_distset + (average_distset - distset[i])

    return distset

def create_new_distance_matrix(distances, lonliness):
    num_cities = len(distances)
    new_distances = [[0] * num_cities for _ in range(num_cities)]

    for i in range(num_cities):
        for j in range(num_cities):
            new_distances[i][j] = ((num_cities * distances[i][j]) + lonliness[j]) / 2

    return new_distances

def calculate_tour_cost(tour, distances):
    cost = 0
    for i in range(len(tour) - 1):
        cost += distances[tour[i]][tour[i + 1]]
    return cost
"""

def double_ended_nearest_loneliest_neighbor(tour,visited, num_cities, old_distances, distances):
    start_time = time.time()
    # If the tour is empty, initialize it with the nearest neighbor algorithm
    if len(tour) == 0:

        visited = [False] * num_cities
        # Initialize variables to track the minimum distance and the cities
        min_distance = float('inf')
        start_city = None
        next_city = None

        # Step 1: Find the shortest edge connecting two cities
        for i in range(num_cities):
            for j in range(num_cities):
                if (i != j) and (distances[i][j] < min_distance):
                    min_distance = distances[i][j]
                    start_city = i
                    next_city = j

        # Add the starting city and its nearest neighbor to the tour
        tour.append(start_city)
        tour.append(next_city)
        visited[start_city] = True
        visited[next_city] = True

    # If there are still cities to visit
    elif len(tour) < num_cities:

        # Step 2: Consider the nodes closer to each of the route’s ends
        # and add to the tour the one closer to the respective endpoint
        start_min_distance = float('inf')
        end_min_distance = float('inf')
        start_city = tour[0]
        end_city = tour[-1]

        for i in range(num_cities):
            if not visited[i]:
                if distances[i][tour[0]] < start_min_distance:
                    start_min_distance = distances[i][tour[0]]
                    start_city = i
                if distances[i][tour[-1]] < end_min_distance:
                    end_min_distance = distances[i][tour[-1]]
                    end_city = i

        # Choose the closer city and add it to the tour
        if start_min_distance <= end_min_distance:
            tour.insert(0, start_city)
            visited[start_city] = True
        else:
            tour.append(end_city)
            visited[end_city] = True
    else:
    # Step 4: Return to the starting node by adding it to the end of the route
      tour.append(tour[0])


    cost = calculate_tour_cost(tour, old_distances)
    end_time = time.time()
    elapsed_time = end_time - start_time

    return tour, visited, cost, elapsed_time


NumberofCities = len(distances)
tour = []
visited = []

# Step 2: Calculate loneliness of each city
lonliness = calculate_loneliness(distances)

# Step 3: Create new distance matrix
new_distances = create_new_distance_matrix(distances, lonliness)

#tour, visited, cost, elapsed_time = nearest_neighbor(tour, visited, NumberofCities, new_distances)

while len(tour) <= NumberofCities:
    tour, visited, cost, elapsed_time = double_ended_nearest_loneliest_neighbor(tour, visited, NumberofCities, distances ,new_distances)

print(f"Elapsed Time: {elapsed_time} seconds")
print("Optimal Tour:", tour)
print("Tour Cost:", cost)

Elapsed Time: 7.152557373046875e-06 seconds
Optimal Tour: [40, 7, 39, 30, 37, 8, 0, 15, 21, 2, 33, 1, 28, 4, 47, 41, 25, 3, 9, 23, 44, 34, 31, 38, 24, 13, 22, 10, 11, 14, 32, 45, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 46, 20, 12, 40]
Tour Cost: 41920.00604175645


# 2-OPT

In [ ]:
def calculate_distance(city1, city2, adjacency_matrix):
    """Calculates the distance between two cities using the adjacency matrix."""
    return adjacency_matrix[city1][city2]

def evalue_path(curr_path, distance_matrix):
    distance = 0
    for i in range(len(curr_path) - 1):
        if curr_path[i + 1] == -1:
            break
        #print(curr_path)
        distance += distance_matrix[curr_path[i]][curr_path[i + 1]]
    return distance

def two_opt(route, adjacency_matrix):

    start_time = time.time()
    """Performs the 2-opt algorithm to improve a given route."""
    if len(route) > 2 :
        improvement = True
        while improvement:
            improvement = False
            for i in range(len(route) - 2):
                for j in range(i + 2, len(route)):
                    if i == 0 and j == len(route) - 1:
                        continue  # Skip swapping start and end points

                    current_distance = calculate_distance(route[i], route[i + 1], adjacency_matrix) + calculate_distance(route[j], route[(j + 1) % len(route)], adjacency_matrix)
                    new_distance = calculate_distance(route[i], route[j], adjacency_matrix) + calculate_distance(route[i + 1], route[(j + 1) % len(route)], adjacency_matrix)

                    if new_distance < current_distance:
                        route[i + 1 : j + 1] = route[j : i : -1]  # Reverse the sub-route
                        improvement = True
    end_time = time.time()
    cost = evalue_path(route, distances)
    elapsed_time = end_time - start_time

    return route, cost, elapsed_time



#circuit, total_cost  = christofides_algorithm(distances)
#initial_route = circuit

tour = list(range(len(distances)))
tour.append(tour[0])
print(tour)
tour, cost, elapsed_time  = two_opt(tour, distances)


print(f"Elapsed Time: {elapsed_time} seconds")
print("Optimized route:", tour)
print("cout ", cost)



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 0]
Elapsed Time: 0.0060880184173583984 seconds
Optimized route: [0, 21, 15, 40, 28, 1, 41, 25, 3, 34, 44, 9, 23, 4, 47, 38, 31, 20, 24, 13, 33, 2, 39, 10, 22, 12, 46, 19, 11, 14, 32, 45, 35, 29, 42, 16, 26, 18, 36, 5, 27, 6, 17, 43, 30, 37, 8, 7, 0]
cout  35329.419573154315


#Recuit Simule

In [ ]:
#Calcule du cout d'un chemin
def total_distance(path, cities):
    distance = 0
    for i in range(len(path) - 1):
        distance += cities[path[i], path[i + 1]]
    distance += cities[path[-1], path[0]]  # Retour à la première ville

    return distance

#Generation du voisin
def generate_neighbor(path):
    i, j = sorted(random.sample(range(len(path)), 2)) #Choisir 2 villes aleatoire
    return path[:i] + path[j:j+1] + path[i+1:j] + path[i:i+1] + path[j+1:] #Permuter entre ces 2 villes


#Algorithme de recuit Simule continue.
#Les parametres sont (matrice d'adjacence, la temperature initiale, coefficient de dimunition de temperature, nombre d'iteration total,
# nombre d'iteration si y'a pas une nouvelle solution, coefficient d'augmentation de temperature )
def simulated_annealing(tour,cities, initial_temperature=10000, cooling_rate=0.88, nb_iter=50000, cooldown_interval=50, cooldown_factor=1.1):
    start_time = time.time()
    #initialisation
    cities = np.array(cities)
    current_path = tour
    #print("Path initial : " , current_path)
    current_distance = total_distance(current_path, cities)
    #print("Sa distance : " , current_distance)
    best_path = current_path.copy()
    best_distance = current_distance
    if len(tour) > 2:
      temperature = initial_temperature
      no_improvement_counter = 0


      for _ in range(nb_iter):
          new_path = generate_neighbor(current_path) #generer un voisin
          new_distance = total_distance(new_path, cities)
          delta_distance = new_distance - current_distance
          if delta_distance < 0 or random.random() < math.exp(-delta_distance / temperature):
            #On prend le voisin si il ameliore la fonction objectif ou avec une certain probabilite
              current_path = new_path
              current_distance = new_distance


              if current_distance < best_distance:
                #Si le voisin ameliore, il devient la meilleure solution
                  best_path = current_path
                  best_distance = current_distance

              temperature *= cooling_rate #demunition de temperature
              no_improvement_counter = 0

          else:
            #Si on prend pas le voisin, augmenter le compteur
              no_improvement_counter += 1

          if no_improvement_counter >= cooldown_interval:
            #Si aucune solution n’est acceptée après cooldown_interval, incrémenter la valeur de la température
              temperature *= cooldown_factor
              no_improvement_counter = 0 #reinitialiser le compteur

    end_time = time.time()
    elapsed_time = end_time - start_time
    return best_path, best_distance,elapsed_time


# Exemple d'utilisation
#cities = np.array(cities)
#start_time = time.time()
best_path, best_distance, elapsed_time = simulated_annealing(tour,distances)
print("Meilleur chemin trouvé:", best_path)
print("Distance totale:", best_distance)
print("Time:", elapsed_time)

Meilleur chemin trouvé: [0, 15, 21, 2, 33, 40, 28, 1, 25, 3, 34, 44, 23, 9, 41, 4, 47, 38, 31, 20, 12, 24, 13, 22, 39, 14, 11, 10, 46, 19, 32, 45, 17, 35, 27, 29, 42, 16, 26, 18, 36, 5, 6, 43, 30, 37, 8, 7, 0]
Distance totale: 34495.326824186894
Time: 1.0230398178100586


#Recherche Tabou

In [ ]:
matrice_distances = distances
# Fonction pour calculer le coût d'une solution
def calculer_cout(S, matrice_distances):
    cout = 0
    n = len(S)
    for i in range(n):
        cout += matrice_distances[S[i]][S[(i + 1) % n]]  # Distance entre la ville i et i+1
    return cout

# Fonction pour générer un voisinage (2-opt swap)
def generer_voisinage(solution):
    voisinage = []
    for i in range(1, len(solution) - 1):
        for j in range(i + 1, len(solution)):
            voisin = solution[:]
            voisin[i:j] = voisin[i:j][::-1]
            voisinage.append(voisin)
    return voisinage

# Implémentation de l'algorithme de recherche tabou
def recherche_tabou(tour,matrice_distances, nb_iterations_max=200, taille_liste_tabou=15):
    start_time = time.time()

    # Initialisations
    nb_villes = len(matrice_distances)
    solution_courante = tour
    meilleure_solution = solution_courante[:]
    cout_meilleure_solution = calculer_cout(meilleure_solution, matrice_distances)

    if len(tour) > 2:

      liste_tabou = []
      # Boucle principale
      for iteration in range(nb_iterations_max):
          voisinage = generer_voisinage(solution_courante)
          couts_voisinage = [calculer_cout(voisin, matrice_distances) for voisin in voisinage]
          # Sélection du meilleur voisin non tabou ou qui respecte le critère d'aspiration
          voisinage_et_couts = sorted([(voisin, cout) for voisin, cout in zip(voisinage, couts_voisinage)], key=lambda x: x[1])
          for voisin, cout_voisin in voisinage_et_couts:
              if (tuple(voisin), cout_voisin) not in liste_tabou or cout_voisin < cout_meilleure_solution:
                  solution_courante = voisin
                  cout_solution_courante = cout_voisin
                  break

          # Mise à jour de la liste tabou
          if len(liste_tabou) >= taille_liste_tabou:
              liste_tabou.pop(0)  # Supprime le plus ancien si la taille maximale est atteinte
          liste_tabou.append((tuple(solution_courante), cout_solution_courante))

          # Mise à jour de la meilleure solution
          if cout_solution_courante < cout_meilleure_solution:
              meilleure_solution = solution_courante[:]
              cout_meilleure_solution = cout_solution_courante

          #print(f"Iteration {iteration}: Cout actuel = {cout_solution_courante}, Meilleur cout = {cout_meilleure_solution}")
    end_time = time.time()
    elapsed_time = end_time - start_time
    return meilleure_solution, cout_meilleure_solution,elapsed_time






# Lancement de l'algorithme
meilleure_solution, cout_meilleure_solution,elapsed_time = recherche_tabou(tour,matrice_distances)


print("Meilleure solution trouvée:")
print(meilleure_solution)
print("Coût de la meilleure solution:")
print(cout_meilleure_solution)
print(f"Temps d'exécution: {elapsed_time} secondes")


Meilleure solution trouvée:
[0, 7, 8, 37, 30, 43, 17, 6, 27, 5, 36, 18, 26, 16, 42, 29, 35, 45, 32, 14, 11, 19, 46, 12, 22, 10, 39, 2, 33, 13, 24, 20, 31, 38, 47, 4, 23, 9, 44, 34, 3, 25, 41, 1, 28, 40, 15, 21, 0]
Coût de la meilleure solution:
35329.4195731543
Temps d'exécution: 3.0169875621795654 secondes


# Selection par roulette haut niveau

Le principe de cette méthode est que chaque heuristique de bas niveau est pondérée par un poids spécifique. L'algorithme est comme suit:
1. On commence par initialiser le tableau des scores (coefficients).
2. Ensuite, on choisit une heuristique aléatoirement en tenant compte des coefficients.
3. Enfin, on applique l'heuristique pour construire la tournée complète.

On répète l'algorithme un certain nombre d'itérations et on choisit la meilleure tournée obtenue.

In [ ]:
"""
distances = [
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 15, 10],
    [20, 25, 15, 0, 5],
    [25, 30, 10, 5, 0]
]
"""

NumberofCities = len(distances)
tour = []
visited = []

# Step 2: Calculate loneliness of each city
lonliness = calculate_loneliness(distances)

# Step 3: Create new distance matrix
new_distances = create_new_distance_matrix(distances, lonliness)


heuristics = ['H1', 'H2', 'H3', 'H4', 'H5']

# Corresponding coefficients
alpha_v = 0.1
beta_v = 0.2
gamma_v = 0.4
lambda_v = 0.3
delta_v = 0.05
coefficients = [0.1, 0.2, 0.4, 0.3, 0.05]


max_iterations = 10
best_cost = float('inf')
best_tour = []
best_heuristic_path = []
best_time_s = 0.0

# Loop to construct the solution
for _ in range(max_iterations):
    tour = []
    visited = []
    heuristic_path = []
    time_s = 0.0

    while len(tour) <= NumberofCities:
        #print (tour)
        selected_heuristic = random.choices(heuristics, weights=coefficients, k=1)[0]
        if selected_heuristic == 'H1':
            # Action for H1
            tour, visited, cost, elapsed_time = nearest_neighbor(tour, visited, NumberofCities, distances)
            #print("Applying Heuristic H1")
        elif selected_heuristic == 'H2':
            # Action for H2
            tour, visited, cost, elapsed_time = nearest_loneliest_neighbor(tour, visited, NumberofCities, distances ,new_distances)
            #print("Applying Heuristic H2")
        elif selected_heuristic == 'H3':
            # Action for H3
            tour, visited, cost, elapsed_time = double_ended_nearest_neighbor(tour, visited, NumberofCities, distances)
            #print("Applying Heuristic H3")
        elif selected_heuristic == 'H4':
            # Action for H4
            tour, visited, cost, elapsed_time = double_ended_nearest_loneliest_neighbor(tour, visited, NumberofCities, distances ,new_distances)
            #print("Applying Heuristic H4")
        elif selected_heuristic == 'H5':
            # Action for H5
            tour, cost, elapsed_time = two_opt(tour, distances)
            #print("Applying Heuristic H4")
        else:
            # Default action if heuristic is not recognized
            print("Unknown Heuristic")
        heuristic_path.append(selected_heuristic)
        time_s += elapsed_time

    if cost < best_cost:
        best_cost = cost
        best_tour = tour
        best_heuristic_path = heuristic_path
        best_time_s = time_s



print("Heuristic Path:", best_heuristic_path)
print(f"Elapsed Time: {best_time_s} seconds")
print("Optimal Tour:", best_tour)
print("Tour Cost:", best_cost)

Heuristic Path: ['H3', 'H4', 'H1', 'H5', 'H2', 'H1', 'H3', 'H4', 'H2', 'H2', 'H2', 'H3', 'H3', 'H4', 'H4', 'H3', 'H2', 'H2', 'H3', 'H4', 'H4', 'H2', 'H2', 'H5', 'H3', 'H5', 'H3', 'H3', 'H3', 'H4', 'H4', 'H2', 'H4', 'H4', 'H4', 'H4', 'H2', 'H3', 'H1', 'H4', 'H5', 'H3', 'H4', 'H3', 'H3', 'H4', 'H2', 'H1', 'H4', 'H4', 'H3', 'H3']
Elapsed Time: 0.0036535263061523438 seconds
Optimal Tour: [9, 41, 1, 28, 4, 47, 33, 40, 2, 21, 15, 0, 7, 8, 37, 30, 43, 17, 6, 27, 5, 36, 18, 26, 16, 42, 29, 35, 45, 32, 19, 14, 39, 11, 10, 22, 46, 20, 12, 13, 24, 38, 31, 23, 44, 34, 25, 3, 9]
Tour Cost: 35294.700766122405


# ACO haut niveau

In [ ]:
# Fonction pour calculer la longueur du chemin
def tour_length(tour, distances):
    total_distance = 0
    for i in range(len(tour) - 1):
        total_distance += distances[tour[i]][tour[i+1]]
    return total_distance

# Fonction pour initialiser les phéromones sur chaque chemin
def initialize_pheromones(num_heuristic,v_init):
    pheromones = [[v_init] * num_heuristic for _ in range(num_heuristic)]
    return pheromones

def initialize_visibility(num_heuristic):
    visibility = [1] * num_heuristic
    return visibility

# Algorithme ACO

def ant_colony_optimization(low_l_heuristics, visibility, distances,new_distances,  num_ants, num_iterations,pheromones, alpha, beta, Q, evaporation_rate):
    num_heuristic = len(visibility)
    NumberofCities = len(distances)

    best_tour = []
    best_solution = None
    best_distance = float('inf')
    best_time_s = 0

    for _ in range(num_iterations):
        pheromones_local= initialize_pheromones(num_heuristic, 0.0)


        for ant in range(num_ants):

            tour = []
            visited = []
            time_s = 0

            current_heuristic = random.randint(0, num_heuristic - 1)
            solution = [current_heuristic] # la solution de l'heuristic [H1,H2,H3]


            # A chaque fois on augmente la visibilité par rapport à elapsed_time en favorisant l'heuristique avec un temps CPU qui est petit
            # Au debut on demarre avec 1 et on augmente
            # Ensuite les heuristique les moins utilisé vont avoir un temps petit ce qui va aider pour la diversification heuristicienne car les autres qui n'ont jamais été utilisé vont avoir plus de chance pour etre utilisé
            # de plus on va pas bloquer dans une heuristique qui a le moins de temps CPU
            while len(tour) < NumberofCities + 1 : #ou egal pour utilisé en dernier la derniere heuristique par amelioration
                probabilities = []
                total_pheromone = sum(pheromones[current_heuristic][neighbor] ** alpha * (1 / visibility[neighbor]) ** beta for neighbor in range(num_heuristic) if neighbor in low_l_heuristics)
                for neighbor in range(num_heuristic):
                        pheromone = pheromones[current_heuristic][neighbor]
                        probability = (pheromone ** alpha) * ((1 / visibility[neighbor]) ** beta) / total_pheromone
                        #print(neighbor,probability)
                        probabilities.append((neighbor, probability))

                selected_heuristic = random.choices([neighbor for neighbor, _ in probabilities], [prob for _, prob in probabilities])[0]

                #print(tour)
                if selected_heuristic == 0 :
                    # Action for H1
                    tour, visited, cost, elapsed_time = nearest_neighbor(tour, visited, NumberofCities, distances)
                    #print("Applying Heuristic H1")
                elif selected_heuristic == 1:
                    # Action for H2
                    tour, visited, cost, elapsed_time = nearest_loneliest_neighbor(tour, visited, NumberofCities, distances ,new_distances)
                    #print("Applying Heuristic H2")
                elif selected_heuristic == 2:
                    # Action for H3
                    tour, visited, cost, elapsed_time = double_ended_nearest_neighbor(tour, visited, NumberofCities, distances)
                    #print("Applying Heuristic H3")
                elif selected_heuristic == 3:
                    # Action for H4
                    tour, visited, cost, elapsed_time = double_ended_nearest_loneliest_neighbor(tour, visited, NumberofCities, distances ,new_distances)
                    #print("Applying Heuristic H4")
                elif selected_heuristic == 4:
                    # Action for H5
                    tour, cost, elapsed_time = two_opt(tour, distances)  ###########ICI On peut par exemple imposer à ne pas l'executer apres un certain nombre d'execution ie : si on execute 2-opt 10 fois on l'executera plus
                    #print("Applying Heuristic H4")
                else:
                    # Default action if heuristic is not recognized
                    print("Unknown Heuristic")

                time_s += elapsed_time
                solution.append(selected_heuristic)
                current_heuristic = selected_heuristic
                visibility[current_heuristic] += elapsed_time


            #tour.append(tour[0])
            #tour_dist = tour_length(tour, distances)

            if cost < best_distance:
                best_solution = solution
                best_distance = cost
                best_tour = tour
                best_time_s = time_s

            for i in range(len(solution) - 1):
                pheromones_local[solution[i]][solution[i+1]] += Q / cost

        for i in range(num_heuristic):
            for j in range(num_heuristic):
                pheromones[i][j] = (1 - evaporation_rate) * pheromones[i][j] + pheromones_local[i][j]

    return best_solution, best_distance, best_tour, best_time_s

# Exemple d'utilisation
"""
distances = [
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 15, 10],
    [20, 25, 15, 0, 5],
    [25, 30, 10, 5, 0]
]
"""
# Step 2: Calculate loneliness of each city
lonliness = calculate_loneliness(distances)

# Step 3: Create new distance matrix
new_distances = create_new_distance_matrix(distances, lonliness)

num_heuristic = 5
heuristics =  list(range(num_heuristic))
num_cities = len(distances)


num_ants = 5
num_iterations = 20
pheromones = initialize_pheromones(num_heuristic, 0.3)
visibility = initialize_visibility(num_heuristic)
alpha = 1.0
beta = 1.0
Q = 1
evaporation_rate = 0.2

best_solution, best_distance, best_tour, best_time = ant_colony_optimization(heuristics, visibility, distances, new_distances, num_ants, num_iterations, pheromones, alpha, beta, Q, evaporation_rate)
print("Meilleur chemin d'heuristiques trouvé :", best_solution)
print("Longueur du meilleur chemin:", best_distance)
print("Meilleur chemin:", best_tour)
print("Temps de meilleur chemin d'heuristique", best_time)

Meilleur chemin d'heuristiques trouvé : [1, 3, 1, 3, 0, 0, 2, 2, 4, 1, 3, 4, 2, 2, 4, 3, 4, 0, 3, 0, 0, 4, 2, 3, 4, 3, 0, 2, 1, 0, 0, 0, 3, 1, 3, 2, 0, 0, 2, 1, 2, 0, 2, 0, 1, 4, 0, 2, 2, 0, 4, 0, 0, 3, 2, 2, 4, 1]
Longueur du meilleur chemin: 34216.509917855634
Meilleur chemin: [25, 3, 34, 44, 9, 23, 31, 38, 24, 13, 12, 20, 46, 19, 32, 14, 45, 35, 29, 42, 16, 26, 18, 36, 5, 27, 6, 17, 43, 30, 37, 8, 7, 0, 39, 11, 10, 22, 2, 21, 15, 40, 33, 47, 4, 28, 1, 41, 25]
Temps de meilleur chemin d'heuristique 0.028883695602416992


# Parametres à partir du model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import joblib

# Function to test the loaded model
def test_model(model, X_new, target_name):
    y_pred = model.predict(X_new)
    #print(f'Prediction for {target_name}: {y_pred}')
    return y_pred

# Load the trained models
loaded_model_ants = joblib.load('model_ants.pkl')
loaded_model_iterations = joblib.load('model_iterations.pkl')
loaded_model_alpha = joblib.load('model_alpha.pkl')
loaded_model_beta = joblib.load('model_beta.pkl')
loaded_model_init_pheromone = joblib.load('model_init_pheromone.pkl')
loaded_model_evaporation_rate = joblib.load('model_evaporation_rate.pkl')

# Load feature names
feature_names = joblib.load('feature_names.pkl')

# Provided distance matrix
"""
distances = [
    [0.0, 1.6599999999999966, 5.077203954934244, 6.519148717432358, 8.83386665056701, 5.530226035163483, 4.104436623947311, 0.7543208866258478, 1.291239714383043, 3.1522690240523623, 1.281405478371309, 5.0756871455991055, 3.115204648173216, 3.9378801403801003],
    [1.6599999999999966, 0.0, 4.088324840322742, 6.01592054468807, 9.196608070370294, 5.75960068060278, 4.759873947910805, 1.988818744883513, 2.944910864525442, 4.404406884019691, 2.940612181162281, 5.179285665031426, 3.984934127435483, 3.621670885102622],
    [5.077203954934244, 4.088324840322742, 0.0, 2.445178930058085, 6.964883344320987, 3.9960230229566918, 4.496087187766704, 4.734405981746813, 6.14733275494339, 8.223016478154472, 6.008260979684545, 3.368590209568386, 4.64009698174509, 2.009999999999991],
    [6.519148717432358, 6.01592054468807, 2.445178930058085, 0.0, 4.800260409602788, 2.708228203087761, 4.124184767926862, 5.955039882318172, 7.291652761891499, 9.597817460235426, 7.1006830657338815, 2.384386713601633, 4.797707786016145, 2.5850338489079765],
    [8.83386665056701, 9.196608070370294, 6.964883344320987, 4.800260409602788, 0.0, 3.4422376443238196, 4.765081321446678, 8.08600024734108, 8.93109735698811, 11.21457979596204, 8.70112636386807, 4.060369441319346, 5.821039426081909, 5.8013532903969915],
    [5.530226035163483, 5.75960068060278, 3.9960230229566918, 2.708228203087761, 3.4422376443238196, 0.0, 1.8115739013355214, 4.805996254680189, 5.853110284284757, 8.215071515209106, 5.629369414064064, 0.6648308055437824, 2.806153951585692, 2.4286004199950226],
    [4.104436623947311, 4.759873947910805, 4.496087187766704, 4.124184767926862, 4.765081321446678, 1.8115739013355214, 0.0, 3.3504925011108417, 4.185510721524912, 6.513555096872982, 3.9564125164092765, 1.7740913167027172, 1.065692263273032, 2.499059823213521],
    [0.7543208866258478, 1.988818744883513, 4.734405981746813, 5.955039882318172, 8.08600024734108, 4.805996254680189, 3.3504925011108417, 0.0, 1.413541651314164, 3.6429932747673286, 1.2794530081249462, 4.376345507383987, 2.36425463941598, 3.3733810932060484],
    [1.291239714383043, 2.944910864525442, 6.14733275494339, 7.291652761891499, 8.93109735698811, 5.853110284284757, 4.185510721524912, 1.413541651314164, 0.0, 2.3685649663878787, 0.23000000000000043, 5.518378384996806, 3.1200320511174233, 4.730010570812711],
    [3.1522690240523623, 4.404406884019691, 8.223016478154472, 9.597817460235426, 11.21457979596204, 8.215071515209106, 6.513555096872982, 3.6429932747673286, 2.3685649663878787, 0.0, 2.5880494585691394, 7.886811776630656, 5.450660510433576, 7.016159918359903],
    [1.281405478371309, 2.940612181162281, 6.008260979684545, 7.1006830657338815, 8.70112636386807, 5.629369414064064, 3.9564125164092765, 1.2794530081249462, 0.23000000000000043, 2.5880494585691394, 0.0, 5.301339453383452, 2.890830330545187, 4.547801666739655],
    [5.0756871455991055, 5.179285665031426, 3.368590209568386, 2.384386713601633, 4.060369441319346, 0.6648308055437824, 1.7740913167027172, 4.376345507383987, 5.518378384996806, 7.886811776630656, 5.301339453383452, 0.0, 2.6122212769977917, 1.7681911661356111],
    [3.115204648173216, 3.984934127435483, 4.64009698174509, 4.797707786016145, 5.821039426081909, 2.806153951585692, 1.065692263273032, 2.36425463941598, 3.1200320511174233, 5.450660510433576, 2.890830330545187, 2.6122212769977917, 0.0, 2.66810794384335],
    [3.9378801403801003, 3.621670885102622, 2.009999999999991, 2.5850338489079765, 5.8013532903969915, 2.4286004199950226, 2.499059823213521, 3.3733810932060484, 4.730010570812711, 7.016159918359903, 4.547801666739655, 1.7681911661356111, 2.66810794384335, 0.0]
]
"""
# Flatten the distance matrix
flattened_distances = [dist for row in distances for dist in row]

# Create a dictionary for the DataFrame
data_dict = {f'distance_{i}': [dist] for i, dist in enumerate(flattened_distances)}
data_dict['num_cities'] = [float(len(distances))]
data_dict['num_heuristics'] = [float(5)]

# Create DataFrame
df = pd.DataFrame(data_dict)

# Reindex to ensure the correct order and presence of all features
df = df.reindex(columns=feature_names, fill_value=0)

# Display DataFrame
print(df)

#num_heuristic = 5
#heuristics = list(range(num_heuristic))
#num_cities = len(distances)

num_ants = int(test_model(loaded_model_ants, df, 'num_ants')[0])
num_iterations = int(test_model(loaded_model_iterations, df, 'num_iterations')[0])
pheromones = initialize_pheromones(num_heuristic, test_model(loaded_model_init_pheromone, df, 'init_pheromone')[0])
alpha = test_model(loaded_model_alpha, df, 'alpha')[0]
beta = test_model(loaded_model_beta, df, 'beta')[0]
evaporation_rate = test_model(loaded_model_evaporation_rate, df, 'evaporation_rate')[0]
Q = 1
visibility = initialize_visibility(num_heuristic)


# Test the loaded models on the new instance
print(f'Prediction for {num_ants}: num_ants')
print(f'Prediction for {num_iterations}: num_iterations')
print(f'Prediction for {pheromones}: pheromones')
print(f'Prediction for {alpha}: alpha')
print(f'Prediction for {beta}: beta')
print(f'Prediction for {evaporation_rate}: evaporation_rate')
print(f'Prediction for {1}: Q')




   distance_0  distance_1   distance_2   distance_3  distance_4  distance_5  \
0         0.0  4726.65315  1204.349202  6362.502102  3656.99125  3129.52089   

    distance_6  distance_7  distance_8   distance_9  ...  distance_5768  \
0  2413.522322  562.304188  462.082244  5653.503339  ...              0   

   distance_5769  distance_5770  distance_5771  distance_5772  distance_5773  \
0              0              0              0              0              0   

   distance_5774  distance_5775  num_cities  num_heuristics  
0              0              0        48.0             5.0  

[1 rows x 5778 columns]
Prediction for 14: num_ants
Prediction for 14: num_iterations
Prediction for [[0.18421340535434175, 0.18421340535434175, 0.18421340535434175, 0.18421340535434175, 0.18421340535434175], [0.18421340535434175, 0.18421340535434175, 0.18421340535434175, 0.18421340535434175, 0.18421340535434175], [0.18421340535434175, 0.18421340535434175, 0.18421340535434175, 0.18421340535434175, 0.1

# ACO haut niveau avec les threads

In [ ]:
# Function to calculate the total distance of a tour
def tour_length(tour, distances):
    total_distance = 0
    for i in range(len(tour) - 1):
        total_distance += distances[tour[i]][tour[i+1]]
    return total_distance

# Function to initialize pheromones on each path
def initialize_pheromones(num_heuristic, v_init):
    pheromones = [[v_init] * num_heuristic for _ in range(num_heuristic)]
    return pheromones

def initialize_visibility(num_heuristic):
    visibility = [1] * num_heuristic
    return visibility


# Worker function for each ant
def ant_worker(ant, num_ants, num_heuristic, NumberofCities, distances, new_distances, visibility, pheromones, alpha, beta, Q, low_l_heuristics, pheromones_local, results_lock, visibility_lock, results):
    tour = []
    visited = []
    time_s = 0

    current_heuristic = random.randint(0, num_heuristic - 1)
    solution = [current_heuristic]

    while len(tour) < NumberofCities + 1:
        probabilities = []
        total_pheromone = sum(pheromones[current_heuristic][neighbor] ** alpha * (1 / visibility[neighbor]) ** beta for neighbor in range(num_heuristic) if neighbor in low_l_heuristics)
        for neighbor in range(num_heuristic):
            pheromone = pheromones[current_heuristic][neighbor]
            probability = (pheromone ** alpha) * ((1 / visibility[neighbor]) ** beta) / total_pheromone
            probabilities.append((neighbor, probability))

        selected_heuristic = random.choices([neighbor for neighbor, _ in probabilities], [prob for _, prob in probabilities])[0]

        if selected_heuristic == 0:
            tour, visited, cost, elapsed_time = nearest_neighbor(tour, visited, NumberofCities, distances)
        elif selected_heuristic == 1:
            tour, visited, cost, elapsed_time = nearest_loneliest_neighbor(tour, visited, NumberofCities, distances, new_distances)
        elif selected_heuristic == 2:
            tour, visited, cost, elapsed_time = double_ended_nearest_neighbor(tour, visited, NumberofCities, distances)
        elif selected_heuristic == 3:
            tour, visited, cost, elapsed_time = double_ended_nearest_loneliest_neighbor(tour, visited, NumberofCities, distances, new_distances)
        elif selected_heuristic == 4:
            tour, cost, elapsed_time = two_opt(tour, distances)
        else:
            print("Unknown Heuristic")

        time_s += elapsed_time
        solution.append(selected_heuristic)
        current_heuristic = selected_heuristic

        # Update visibility globally
        with visibility_lock:
            visibility[current_heuristic] += elapsed_time

    with results_lock:
        results.append((solution, cost, tour, time_s))
        for i in range(len(solution) - 1):
            pheromones_local[solution[i]][solution[i + 1]] += Q / cost

# ACO Algorithm
def ant_colony_optimization(low_l_heuristics, visibility, distances, new_distances, num_ants, num_iterations, pheromones, alpha, beta, Q, evaporation_rate):
    num_heuristic = len(visibility)
    NumberofCities = len(distances)

    best_tour = []
    best_solution = None
    best_distance = float('inf')
    best_time_s = 0

    for _ in range(num_iterations):
        pheromones_local = initialize_pheromones(num_heuristic, 0.0)
        results = []
        results_lock = threading.Lock()
        visibility_lock = threading.Lock()
        threads = []

        for ant in range(num_ants):
            t = threading.Thread(target=ant_worker, args=(ant, num_ants, num_heuristic, NumberofCities, distances, new_distances, visibility, pheromones, alpha, beta, Q, low_l_heuristics, pheromones_local, results_lock, visibility_lock, results))
            threads.append(t)
            t.start()

        for t in threads:
            t.join()

        for solution, cost, tour, time_s in results:
            if cost < best_distance:
                best_solution = solution
                best_distance = cost
                best_tour = tour
                best_time_s = time_s

        for i in range(num_heuristic):
            for j in range(num_heuristic):
                pheromones[i][j] = (1 - evaporation_rate) * pheromones[i][j] + pheromones_local[i][j]

    return best_solution, best_distance, best_tour, best_time_s

# Example usage
"""
distances = [
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 15, 10],
    [20, 25, 15, 0, 5],
    [25, 30, 10, 5, 0]
]
"""
# Step 2: Calculate loneliness of each city
loneliness = calculate_loneliness(distances)

# Step 3: Create new distance matrix
new_distances = create_new_distance_matrix(distances, loneliness)

num_heuristic = 5
heuristics = list(range(num_heuristic))
num_cities = len(distances)

"""
pheromones = initialize_pheromones(num_heuristic, 0.3)
num_ants = 5
num_iterations = 20
visibility = initialize_visibility(num_heuristic)
alpha = 1.0
beta = 1.0
Q = 1
evaporation_rate = 0.2
"""
best_solution, best_distance, best_tour, best_time = ant_colony_optimization(heuristics, visibility, distances, new_distances, num_ants, num_iterations, pheromones, alpha, beta, Q, evaporation_rate)
print("Best heuristic path found:", best_solution)
print("Length of the best path:", best_distance)
print("Best tour:", best_tour)
print("Time of best heuristic path:", best_time)


Best heuristic path found: [0, 2, 1, 2, 3, 3, 2, 3, 3, 1, 0, 1, 0, 0, 2, 2, 3, 0, 0, 0, 3, 0, 1, 0, 1, 0, 0, 2, 0, 2, 3, 1, 0, 0, 1, 2, 3, 0, 0, 0, 3, 2, 4, 3, 2, 3, 1, 3, 1, 4, 1]
Length of the best path: 34441.12855333484
Best tour: [39, 14, 11, 32, 45, 35, 29, 42, 16, 26, 18, 36, 5, 27, 6, 17, 43, 30, 37, 8, 7, 0, 15, 21, 2, 33, 40, 28, 1, 25, 3, 34, 44, 9, 23, 41, 4, 47, 38, 31, 20, 46, 19, 10, 12, 24, 13, 22, 39]
Time of best heuristic path: 0.009356021881103516


# Test automatisé et sauvegarde dans un fichier JSON


In [ ]:
def calculate_distance_for_file(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def read_tsp_file(file_path):
    coordinates = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        node_coord_section_index = lines.index('NODE_COORD_SECTION\n')
        for line in lines[node_coord_section_index + 1:]:
            if line.strip() == 'EOF':
                break
            parts = line.strip().split()
            node_id = int(parts[0])
            x, y = map(float, parts[1:])
            coordinates[node_id] = (x, y)
    return coordinates


def create_distance_matrix(coordinates):
    num_cities = len(coordinates)
    distance_matrix = [[0] * num_cities for _ in range(num_cities)]

    for i in range(1, num_cities + 1):
        for j in range(1, num_cities + 1):
            distance_matrix[i - 1][j - 1] = calculate_distance_for_file(coordinates[i], coordinates[j])

    return distance_matrix


def run_experiments(distances, new_distances,num_cities,heuristics_names,num_heuristics, alpha_values, beta_values, Q_values, evaporation_rates, num_ants_values, num_iterations_values,filename):
    results = []
    for num_ants in num_ants_values:
        for num_iterations in num_iterations_values:
              for evaporation_rate in evaporation_rates:
                  for alpha in alpha_values:
                      for beta in beta_values:
                        for init_V in pheremone_init_V:
                           pheromones = initialize_pheromones(num_heuristics, init_V)
                           visibility = initialize_visibility(num_heuristics)
                           heuristics = list(range(num_heuristics))
                           best_solution, best_distance, best_tour, best_time = ant_colony_optimization(heuristics, visibility, distances, new_distances, num_ants, num_iterations, pheromones, alpha, beta, Q, evaporation_rate)

                           result = {
                              'num_ants': num_ants,
                              'num_iterations': num_iterations,
                              'alpha': alpha,
                              'beta': beta,
                              'init_pheromone' : init_V,
                              'Q': Q,
                              'evaporation_rate': evaporation_rate,
                              'num_ants': num_ants,
                              'num_iterations': num_iterations,
                              'best_solution': best_solution,
                              'best_distance': best_distance,
                              'best_tour': best_tour,
                              'best_time': best_time
                          }
                           results.append(result)

    # Read existing data from the file
    execution = {
        "instance": distances,
        "num_cities": num_cities,
        "heuristics": heuristics_names,
        "num_heuristic": num_heuristics,
        "results": results
    }

    # Check if the file exists and is not empty
    if os.path.exists(filename) and os.path.getsize(filename) > 0:
        with open(filename, 'r+') as f:
            f.seek(0, os.SEEK_END)
            f.seek(f.tell() - 1, os.SEEK_SET)
            f.truncate()
            f.write(',\n' + json.dumps(execution, indent=4) + ']')
    else:
        with open(filename, 'w') as f:
            json.dump([execution], f, indent=4)

#For heuristics
heuristics_names = ['NN','NLN','DNN','DNLN','2OPT']
num_heuristics = len(heuristics_names)

pheremone_init_V = [0.1, 0.3]
num_ants_values = [10, 20]
num_iterations_values = [10, 20]
alpha_values = [1.0, 2.0]
beta_values = [1.0, 2.0]
Q_values = [1]
evaporation_rates = [0.1, 0.2]

#file_sources = ["berlin52.tsp", "att48.tsp", "eil51.tsp", "burma14.tsp" , "eil76.tsp"]
file_sources = ["att48.tsp","burma14.tsp"]

filename = 'aco_results.json'

for file_source in file_sources:
  coordinates = read_tsp_file(file_source)
  distances = create_distance_matrix(coordinates)
  loneliness = calculate_loneliness(distances)
  new_distances = create_new_distance_matrix(distances, loneliness)

  num_cities = len(distances)

  run_experiments(distances, new_distances,num_cities,heuristics_names , num_heuristics, alpha_values, beta_values, Q_values, evaporation_rates, num_ants_values, num_iterations_values,filename)
print(f'Results saved to {filename}')

# Test automatisé et sauvegarde dans un fichier CSV


In [ ]:
def calculate_distance_for_file(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def read_tsp_file(file_path):
    coordinates = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        node_coord_section_index = lines.index('NODE_COORD_SECTION\n')
        for line in lines[node_coord_section_index + 1:]:
            if line.strip() == 'EOF':
                break
            parts = line.strip().split()
            node_id = int(parts[0])
            x, y = map(float, parts[1:])
            coordinates[node_id] = (x, y)
    return coordinates


def create_distance_matrix(coordinates):
    num_cities = len(coordinates)
    distance_matrix = [[0] * num_cities for _ in range(num_cities)]

    for i in range(1, num_cities + 1):
        for j in range(1, num_cities + 1):
            distance_matrix[i - 1][j - 1] = calculate_distance_for_file(coordinates[i], coordinates[j])

    return distance_matrix


def write_csv_header(filename):
    header = [
        'instance', 'num_cities', 'heuristics', 'num_heuristics',
        'num_ants', 'num_iterations', 'alpha', 'beta', 'init_pheromone',
        'Q', 'evaporation_rate', 'best_solution', 'best_distance',
        'best_tour', 'best_time'
    ]
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)

def run_experiments(distances, new_distances, num_cities, heuristics_names, num_heuristics, alpha_values, beta_values, Q_values, evaporation_rates, num_ants_values, num_iterations_values, filename):
    if not os.path.exists(filename):
        write_csv_header(filename)

    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        for num_ants in num_ants_values:
            for num_iterations in num_iterations_values:
                for evaporation_rate in evaporation_rates:
                    for alpha in alpha_values:
                        for beta in beta_values:
                            for init_V in pheremone_init_V:
                                pheromones = initialize_pheromones(num_heuristics, init_V)
                                visibility = initialize_visibility(num_heuristics)
                                heuristics = list(range(num_heuristics))
                                best_solution, best_distance, best_tour, best_time = ant_colony_optimization(
                                    heuristics, visibility, distances, new_distances, num_ants, num_iterations, pheromones, alpha, beta, Q, evaporation_rate
                                )

                                result = [
                                    distances, num_cities, heuristics_names, num_heuristics,
                                    num_ants, num_iterations, alpha, beta, init_V, Q,
                                    evaporation_rate, best_solution, best_distance, best_tour, best_time
                                ]
                                writer.writerow(result)


#For heuristics
heuristics_names = ['NN','NLN','DNN','DNLN','2OPT']
num_heuristics = len(heuristics_names)

pheremone_init_V = [0.1, 0.3]
num_ants_values = [10, 20]
num_iterations_values = [10, 20]
alpha_values = [1.0, 2.0]
beta_values = [1.0, 2.0]
Q_values = [1]
evaporation_rates = [0.1, 0.2]

#file_sources = ["berlin52.tsp", "att48.tsp", "eil51.tsp", "burma14.tsp" , "eil76.tsp"]
file_sources = ["berlin52.tsp"]

filename = 'aco_results.csv'

for file_source in file_sources:
  coordinates = read_tsp_file(file_source)
  distances = create_distance_matrix(coordinates)
  loneliness = calculate_loneliness(distances)
  new_distances = create_new_distance_matrix(distances, loneliness)

  num_cities = len(distances)

  run_experiments(distances, new_distances,num_cities,heuristics_names , num_heuristics, alpha_values, beta_values, Q_values, evaporation_rates, num_ants_values, num_iterations_values,filename)
print(f'Results saved to {filename}')

Results saved to aco_results.csv


# Test manuel

In [ ]:
def run_experiments(distances, new_distances, num_heuristics, alpha_values, beta_values, Q_values, evaporation_rates, num_ants_values, num_iterations_values):
    results = []
    for num_ants in num_ants_values:
        for num_iterations in num_iterations_values:
              for evaporation_rate in evaporation_rates:
                  for alpha in alpha_values:
                      for beta in beta_values:
                        for init_V in pheremone_init_V:
                           pheromones = initialize_pheromones(num_heuristics, init_V)
                           visibility = initialize_visibility(num_heuristics)
                           heuristics = list(range(num_heuristics))
                           best_solution, best_distance, best_tour, best_time = ant_colony_optimization(heuristics, visibility, distances, new_distances, num_ants, num_iterations, pheromones, alpha, beta, Q, evaporation_rate)

                           result = {
                              'num_ants': num_ants,
                              'num_iterations': num_iterations,
                              'alpha': alpha,
                              'beta': beta,
                              'init_pheromone' : init_V,
                              'Q': Q,
                              'evaporation_rate': evaporation_rate,
                              'num_ants': num_ants,
                              'num_iterations': num_iterations,
                              'best_solution': best_solution,
                              'best_distance': best_distance,
                              'best_tour': best_tour,
                              'best_time': best_time
                          }
                           results.append(result)

    return results

# Exemple d'utilisation
"""
distances = [
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 15, 10],
    [20, 25, 15, 0, 5],
    [25, 30, 10, 5, 0]
]
"""
# Step 2: Calculate loneliness of each city
loneliness = calculate_loneliness(distances)

# Step 3: Create new distance matrix
new_distances = create_new_distance_matrix(distances, loneliness)

num_heuristics = 5

alpha_values = [3, 4, 5]
beta_values = [3, 4, 5]
Q_values = [1]
evaporation_rates = [0.1, 0.2, 0.3]
num_ants_values = [5, 10, 20]
num_iterations_values = [20, 40, 60]
pheremone_init_V = [0.1, 0.3, 0.5]


results = run_experiments(distances, new_distances, num_heuristics, alpha_values, beta_values, Q_values, evaporation_rates, num_ants_values, num_iterations_values)

for result in results:
    print(f"alpha: {result['alpha']}, beta: {result['beta']}, Q: {result['Q']}, evaporation_rate: {result['evaporation_rate']}, num_ants: {result['num_ants']}, num_iterations: {result['num_iterations']}, init_pheromone: {result['init_pheromone']}")
    print(f"Best solution: {result['best_solution']}")
    print(f"Best distance: {result['best_distance']}")
    print(f"Best tour: {result['best_tour']}")
    print(f"Best time: {result['best_time']}")
    print()